# Predicting Hepatotoxicity Using ToxCast in Vitro Bioactivity and Chemical Structure

http://pubs.acs.org/doi/abs/10.1021/tx500501h

DOI: 10.1021/tx500501h

In [10]:
import numpy as np
import pylab as pl

from matplotlib.colors import ListedColormap
#from matplotlib_venn import venn2,venn3
from sklearn.cross_validation import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
import pandas as pd
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn import cross_validation
from sklearn.feature_selection import RFE
from itertools import *
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble, lda)
from time import time
from sklearn.metrics import precision_score,recall_score,accuracy_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.cross_validation import StratifiedKFold, permutation_test_score,KFold
from sklearn.cluster import KMeans

import warnings
warnings.simplefilter('ignore')
import os
import subprocess
import warnings

import pandas as pd
import re

In [13]:
# This function provides a dump of SysInfo if you're running it under ms windows
# http://stackoverflow.com/questions/467602/how-can-i-read-system-information-in-python-on-windows
def SysInfo():
    values  = {}
    cache   = os.popen2("SYSTEMINFO")
    source  = cache[1].read()
    sysOpts = ["Host Name", "OS Name", "OS Version", "Product ID", "System Manufacturer", "System Model", "System type", "BIOS Version", "Domain", "Windows Directory", "Total Physical Memory", "Available Physical Memory", "Logon Server"]

    for opt in sysOpts:
        values[opt] = [item.strip() for item in re.findall("%s:\w*(.*?)\n" % (opt), source, re.IGNORECASE)][0]
    return values

# Just to confirm this code is running on a stock EPA laptop 
SysInfo()

{'Available Physical Memory': '2,440 MB',
 'BIOS Version': 'Dell Inc. A10, 6/26/2014',
 'Domain': 'aa.ad.epa.gov',
 'Host Name': 'LZ26266ISHAH',
 'Logon Server': '\\\\V1818TNCYT402',
 'OS Name': 'Microsoft Windows 7 Enterprise',
 'OS Version': '6.1.7601 Service Pack 1 Build 7601',
 'Product ID': '00392-918-5000002-85066',
 'System Manufacturer': 'Dell Inc.',
 'System Model': 'Latitude E7240',
 'System type': 'x64-based PC',
 'Total Physical Memory': '8,098 MB',
 'Windows Directory': 'C:\\windows'}

In [14]:
# Here's the main code -- a little verbose 

# Store all classifiers in a dictionary
Classifiers1 = dict(LDA = LDA(),
                    NB= GaussianNB(), 
                    KNN1=KNeighborsClassifier(algorithm='auto',n_neighbors=3), 
                    SVCL0=SVC(kernel='linear'), 
                    SVCR0=SVC(C=1.0, cache_size=400, class_weight=None, coef0=0.0, degree=3,
                              gamma=10.0, kernel='rbf', max_iter=-1, probability=False, shrinking=True,
                              tol=0.0001, verbose=False),
                    CART0=DecisionTreeClassifier(max_depth=10))

DataSet = {}
Markers = {}


def selFeatsFilter(A,b,top=5):
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(A, b)
    scores = -np.log10(selector.pvalues_)
    scores[np.isnan(scores)]=0
    np.argsort(scores)
    if top>len(scores): top=len(scores)
    return np.argsort(scores)[-top:]

def myScore(Y_test,Y_pred):
    X = dict(TP=np.sum(np.logical_and(Y_pred==1, Y_test==1)),
             TN=np.sum(np.logical_and(Y_pred==0, Y_test==0)),
             FP=np.sum(np.logical_and(Y_pred==1, Y_test==0)),
             FN=np.sum(np.logical_and(Y_pred==0, Y_test==1)))
    for i,v in X.iteritems(): v *= 1.0
    TP,TN,FP,FN = X['TP'],X['TN'],X['FP'],X['FN']
    return dict(cm=X,acc=1.0*(TP+TN)/(TP+TN+FP+FN),sens=1.0*TP/(TP+FN),spec=1.0*TN/(TN+FP))

def countPosNeg(X):
    n0=np.sum(X==0)
    n1=np.sum(X==1)
    if n1>n0: 
        return 1
    else:
        return 0

def trainAndTestClassifiers(X,Y,I_test,I_train,n_desc,dbg=False,
                            TOPD=Markers,CLSF=Classifiers1):
    X_test = X.ix[I_test,:]
    X_train = X.ix[I_train,:]
    Y_test = Y.ix[I_test]
    Y_train = Y.ix[I_train]
    F_best = selFeatsFilter(X_train,Y_train,n_desc)
    Perf_i   =[]
    Vote   =pd.DataFrame(np.zeros((len(I_test),len(CLSF))),columns=CLSF.keys())
    
    if dbg: print "Train",X_train.shape," Test",X_test.shape," n_desc",len(F_best)
    for nm,clf in CLSF.iteritems():
        if dbg: print nm
        try:
            if dbg: print " Training"
            clf.fit(X_train.ix[:,F_best], Y_train)
            if dbg: print " Testing"
            Y_pred = clf.predict(X_test.ix[:,F_best])
            Vote[nm]=Y_pred
            if dbg: print " Calc score"
            P_i = myScore(Y_test,Y_pred)
            if dbg: print " Make results"
            P = dict(cl=nm,sens=P_i['sens'],spec=P_i['spec'],
                     bacc=0.5*(P_i['sens']+P_i['spec']),
                     prec=precision_score(Y_test,Y_pred),
                     n_pos=np.sum(Y_train==1),n_neg=np.sum(Y_train==0),
                     n_desc=n_desc,
                     acc=accuracy_score(Y_test,Y_pred))
                
            Perf_i.append(P)
                             
        except:
            print "   %s Failed!" % nm
        
    # Now get the majority vote for the class
    Y_pred=Vote.apply(countPosNeg,axis=1)
    P_i = myScore(Y_test,Y_pred)
    P = dict(cl='ENSMB',sens=P_i['sens'],spec=P_i['spec'],
             bacc=0.5*(P_i['sens']+P_i['spec']),
             prec=precision_score(Y_test,Y_pred),
             n_pos=np.sum(Y_train==1),n_neg=np.sum(Y_train==0),
             n_desc=n_desc,
             acc=accuracy_score(Y_test,Y_pred))
    Perf_i.append(P)

    return pd.DataFrame(Perf_i)


def resampleBalCVLearn(n_pos,n_neg,trial=1,
                       n_desc_min=5,n_desc_max=70,n_desc_step=5,
                       K=10,DS=DataSet,TOPD=Markers,
                       CLSF=Classifiers1,dbg=False):
    """
    Creates a balanced undersampled subset of the data for K-fold CV testing n_reps times 
    Requires: DS (which is pushed by dview)

    """
    Data = DS['Data']
    Y_pos = Data[DS['pos_cls']]
    X_pos = Data[DS['Desc']].ix[Y_pos==1,:]
    N_pos = np.sum(Y_pos[Y_pos==1])
    
    Y_neg = Data[DS['neg_cls']]
    X_neg = Data[DS['Desc']].ix[Y_neg==1,:]
    N_neg = np.sum(Y_neg[Y_neg==1])

    I_pos = np.random.randint(0,N_pos,n_pos)
    X_pi = X_pos.ix[I_pos,:]
    X_pi['out']=1

    I_neg = np.random.randint(0,N_neg,n_neg)
    X_ni = X_neg.ix[I_neg,:]
    X_ni['out']=0

    X = pd.concat((X_pi,X_ni))
    Y = X['out']
    X = X.drop('out',axis=1)
    
    Perf_cv = []
    
    #pid=os.getpid()

    SKF = StratifiedKFold(Y,n_folds=K)
    i_step = 0
    for n_desc in range(n_desc_min,n_desc_max,n_desc_step):
        for I_train,I_test in SKF:
            i_step += 1
            if dbg: print 'Data',X.shape
            P_df=trainAndTestClassifiers(X,Y,I_test,I_train,n_desc,
                                         TOPD=Markers,CLSF=Classifiers1,dbg=dbg)
            P_df['n_obs']=n_pos+n_neg
            P_df['n_pos']=n_pos
            P_df['n_neg']=n_neg
            P_df['dtp'] = DS['dtp']
            P_df['tox'] = DS['tox']
            P_df['cvk'] = K
            P_df['itrl']= "%d-%d" % (trial,i_step)
            Perf_cv += P_df.T.to_dict().values()
        
    return Perf_cv

# The rest is just to make a summary table of performance results from all the replicates

C1=pd.MultiIndex.from_tuples(
[('n_desc', 'bio'),
 ('n_desc', 'chm'),
 ('n_desc', 'bc'),
 ('bacc', 'bio'),
 ('bacc', 'chm'),
 ('bacc', 'bc'),
 ('sens', 'bio'),
 ('sens', 'chm'),
 ('sens', 'bc'),
 ('spec', 'bio'),
 ('spec', 'chm'),
 ('spec', 'bc')])
ii_tox=0
ii_dtp=1
ii_cl =2
ii_cvk=3
ii_nds=4
ii_nob=5


def buildMSTable1(P_st,cols=C1):
    X= P_st.groupby(level=['tox','dtp','cl']).max()
    Tox=set([i[0] for i in X.index])
    Dtp=set([i[1] for i in X.index])
    CL =set([i[2] for i in X.index])
    Res=[]
    for tox in Tox:
        for dtp in Dtp:
            for cl in CL:
                Bacc=X.xs((tox,dtp,cl))['bacc']
                Y = P_st.xs((tox,dtp,cl)).reset_index()[['n_desc','bacc']]
                Y1 = Y[(Y['bacc']==Bacc)].n_desc
                #print tox,dtp,cl,Bacc,int(Y1)
                Res.append(dict(tox=tox,dtp=dtp,cl=cl,n_desc=int(list(Y1)[0])))
    Res_df = pd.DataFrame(Res).set_index(['tox','dtp','cl'])
    X = pd.merge(X,Res_df,left_index=True,right_index=True)
    #X['bacc2'] = X.apply(lambda x: "%3.2f(%d)"%(x[1],x[-1]),axis=1)

    T1 = np.round(pd.pivot_table(X.reset_index(),index=['tox','cl'],columns=['dtp'],values=['bacc','n_desc','sens','spec']),decimals=2)
    return T1[cols]

def buildMSTable2(P_mn,P_sd,cols=C1):
    IMB_mn = buildMSTable1(P_mn)
    IMB_sd = buildMSTable1(P_sd)

    if len(cols)>0: 
        C1 = cols 
    else:
        C1 = IMB_mn.columns
        
    R1=IMB_mn.index
    IMB_res = pd.DataFrame(columns=C1,index=R1)

    for i in [x for x in C1 if x[0]=='n_desc']:
        IMB_res[i] = IMB_mn[i]

    for r in R1:
        for c in [c for c in C1 if c[0]!='n_desc']:
            IMB_res.ix[r,c] = "%3.2f (%3.2f)" % (IMB_mn.ix[r,c],IMB_sd.ix[r,c]) 

    return IMB_res

def buildMSTable11(P_st,cols=C1):
    X= P_st.groupby(level=['tox','dtp']).max()
    Tox=set([i[0] for i in X.index])
    Dtp=set([i[1] for i in X.index])
    Res=[]
    for tox in Tox:
        for dtp in Dtp:
            Bacc=X.xs((tox,dtp))['bacc']
            Y = P_st.xs((tox,dtp)).reset_index()[['n_desc','bacc']]
            Y1 = Y[(Y['bacc']==Bacc)].n_desc
            #print tox,dtp,cl,Bacc,int(Y1)
            Res.append(dict(tox=tox,dtp=dtp,n_desc=int(list(Y1)[0])))
    Res_df = pd.DataFrame(Res).set_index(['tox','dtp'])
    X = pd.merge(X,Res_df,left_index=True,right_index=True)
    #X['bacc2'] = X.apply(lambda x: "%3.2f(%d)"%(x[1],x[-1]),axis=1)

    T1 = np.round(pd.pivot_table(X.reset_index(),index=['tox'],columns=['dtp'],values=['bacc','n_desc','sens','spec']),decimals=2)
    return T1[cols]

def buildMSTable21(P_mn,P_sd,cols=C1):
    IMB_mn = buildMSTable11(P_mn)
    IMB_sd = buildMSTable11(P_sd)

    if len(cols)>0: 
        C1 = cols 
    else:
        C1 = IMB_mn.columns
    
    R1=IMB_mn.index
    IMB_res = pd.DataFrame(columns=C1,index=R1)

    for i in [x for x in C1 if x[0]=='n_desc']:
        IMB_res[i] = IMB_mn[i]

    for r in R1:
        for c in [c for c in C1 if c[0]!='n_desc']:
            IMB_res.ix[r,c] = "%3.2f (%3.2f)" % (IMB_mn.ix[r,c],IMB_sd.ix[r,c]) 

    return IMB_res



In [15]:
import bz2

In [16]:
# The data file integrates chm, bio and tox data
import bz2
DF = pd.read_csv(bz2.BZ2File('liv-tox-pred-1.csv.bz2','r'))
DF1 = DF.set_index(['cid','CASRN','name'])

In [17]:
#Figure out the columns for each descriptor
#print DF1.columns
Col_tox = ['Hypertrophy', 'Injury', 'Proliferative.lesions', 'No.liver.lesions']
Col_bio = [i for i in DF1.columns if re.search(r'^APR|^ATG|^ACEA|^NVS|^OT|^Tox21',i,re.I)]
Col_chm = [i for i in DF1.columns if i not in Col_tox+Col_bio]

print "Tox",len(Col_tox)," Bio",len(Col_bio)," Chm",len(Col_chm)

# Now create the different datasets for ML
#
# DataSet:
#   Desc: list of descriptors
#   pos_cls: name of positive class
#   neg_cls: name of negative class
#   dtp: descriptor type bio|chm|bc
#   Data: DataFrame containing all data
#   title: string name for the data set

ML1 = dict()
ML1['Inj-bio']=dict(Desc=Col_bio,pos_cls='Injury',neg_cls='No.liver.lesions',dtp='bio',
                Data=DF1,
                title="Liver Injury ~ Bioactivity")

ML1['Hyp-bio']=dict(Desc=Col_bio,pos_cls='Hypertrophy',neg_cls='No.liver.lesions',dtp='bio',
                Data=DF1,
                title="Liver Hypertrophy ~ Bioactivity")

ML1['Pro-bio']=dict(Desc=Col_bio,pos_cls='Proliferative.lesions',neg_cls='No.liver.lesions',dtp='bio',
                Data=DF1,
                title="Liver Proliferative Lesions ~ Bioactivity")

ML1['Inj-chm']=dict(Desc=Col_chm,pos_cls='Injury',neg_cls='No.liver.lesions',dtp='chm',
                Data=DF1,
                title="Liver Injury ~ Chem")

ML1['Hyp-chm']=dict(Desc=Col_chm,pos_cls='Hypertrophy',neg_cls='No.liver.lesions',dtp='chm',
                Data=DF1,
                title="Liver Hypertrophy ~ Chem")

ML1['Pro-chm']=dict(Desc=Col_chm,pos_cls='Proliferative.lesions',neg_cls='No.liver.lesions',dtp='chm',
                Data=DF1,
                title="Liver Proliferative Lesions ~ Chem")

ML1['Inj-bc']=dict(Desc=Col_chm+Col_bio,pos_cls='Injury',neg_cls='No.liver.lesions',dtp='bc',
                Data=DF1,
                title="Liver Injury ~ Bio + Chem")

ML1['Hyp-bc']=dict(Desc=Col_chm+Col_bio,pos_cls='Hypertrophy',neg_cls='No.liver.lesions',dtp='bc',
                Data=DF1,
                title="Liver Hypertrophy ~ Bio + Chem")

ML1['Pro-bc']=dict(Desc=Col_chm+Col_bio,pos_cls='Proliferative.lesions',neg_cls='No.liver.lesions',dtp='bc',
                Data=DF1,
                title="Liver Proliferative Lesions ~ Bio + Chem")

for tox,V in ML1.iteritems():
    V['tox']=tox.split('-')[0]
    


Tox 4  Bio 127  Chm 730


In [18]:
for tox,DS in ML1.iteritems():
    Data = DS['Data']
    pos  = DS['pos_cls']
    neg  = DS['neg_cls']
    
    print tox,np.sum(Data[pos]==1),np.sum(Data[neg]==1)


Pro-bc 99 463
Pro-chm 99 463
Inj-chm 101 463
Hyp-bio 161 463
Pro-bio 99 463
Hyp-chm 161 463
Inj-bio 101 463
Inj-bc 101 463
Hyp-bc 161 463


(677, 861)

In [19]:
# This is how to run the whole ML strategy   
# 
import time

PERF   = []
TOPDESC={}
i_trl = 0
for tox in ['Hyp-chm','Hyp-bio','Hyp-bc',
            'Inj-chm','Inj-bio','Inj-bc',
            'Pro-chm','Pro-bio','Pro-bc',
            ]:
    DS = ML1[tox]
    Markers = {}
    Res = []
    print "\n>>", DS['title'],len(PERF),time.strftime("%d/%m/%Y %H:%M",time.localtime())

    Data = DS['Data']
    Y_pos = Data[DS['pos_cls']]
    X_pos = Data[DS['Desc']].ix[Y_pos==1,:]
    N_pos = np.sum(Y_pos[Y_pos==1])
 
    for n_pos in range(10,N_pos,10):
        print "  >  No. of pos: %d (res: %d)" % (n_pos,len(Res)),time.strftime("%d/%m/%Y %H:%M",time.localtime())
        # This runs 5 undersampled datasets  -- need at least 20
        for i in range(5):    
            i_trl+=1
            Res += resampleBalCVLearn(n_pos=n_pos,n_neg=n_pos,trial=i_trl,
                                    n_desc_min=10,n_desc_max=70,n_desc_step=5,
                                    K=10,DS=DS,TOPD=Markers,CLSF=Classifiers1,dbg=False)

    PERF += Res
    TOPDESC[tox]=Markers


print "> ML analysis done ",time.strftime("%d/%m/%Y %H:%M",time.localtime())

PERF2_df = pd.DataFrame(PERF)
PERF2_df.set_index(['tox','dtp','cl','itrl','cvk','n_desc','n_pos','n_neg','n_obs'],inplace=True)
# These are variables used further below
TOP_DESC=TOPDESC


>> Liver Hypertrophy ~ Chem 0 04/08/2016 13:35
  >  No. of pos: 10 (res: 0) 04/08/2016 13:35
  >  No. of pos: 20 (res: 4200) 04/08/2016 13:36
  >  No. of pos: 30 (res: 8400) 04/08/2016 13:36
  >  No. of pos: 40 (res: 12600) 04/08/2016 13:37
  >  No. of pos: 50 (res: 16800) 04/08/2016 13:38
  >  No. of pos: 60 (res: 21000) 04/08/2016 13:39
  >  No. of pos: 70 (res: 25200) 04/08/2016 13:40
  >  No. of pos: 80 (res: 29400) 04/08/2016 13:40
  >  No. of pos: 90 (res: 33600) 04/08/2016 13:41
  >  No. of pos: 100 (res: 37800) 04/08/2016 13:42
  >  No. of pos: 110 (res: 42000) 04/08/2016 13:42
  >  No. of pos: 120 (res: 46200) 04/08/2016 13:43
  >  No. of pos: 130 (res: 50400) 04/08/2016 13:44
  >  No. of pos: 140 (res: 54600) 04/08/2016 13:45
  >  No. of pos: 150 (res: 58800) 04/08/2016 13:45
  >  No. of pos: 160 (res: 63000) 04/08/2016 13:46

>> Liver Hypertrophy ~ Bioactivity 67200 04/08/2016 13:47
  >  No. of pos: 10 (res: 0) 04/08/2016 13:47
  >  No. of pos: 20 (res: 4200) 04/08/2016 13:

In [20]:
P2_mn = PERF2_df.groupby(level=['tox','dtp','cl','cvk','n_desc','n_obs']).mean()
P2_sd = PERF2_df.groupby(level=['tox','dtp','cl','cvk','n_desc','n_obs']).std()


In [21]:
# FInd the rows containing the max number of examples
X = P2_mn.reset_index()
N_obs=X[['tox','n_obs']].groupby('tox').max().to_dict()['n_obs']
Y = []
for tox,n_obs in N_obs.iteritems():
    X1 = X[(X['tox']==tox) & (X['n_obs']==n_obs)]
    Y += [X1.irow(i).to_dict() for i in range(X1.shape[0])]
P21_mn = pd.DataFrame(Y)
P21_mn.set_index(['tox','dtp','cl','cvk','n_desc','n_obs'],inplace=True)
P21_mn = P21_mn.groupby(level=['tox','dtp','cl','cvk','n_desc','n_obs']).mean()

X = P2_sd.reset_index()
Y = []
for tox,n_obs in N_obs.iteritems():
    X1 = X[(X['tox']==tox) & (X['n_obs']==n_obs)]
    Y += [X1.irow(i).to_dict() for i in range(X1.shape[0])]
P21_sd = pd.DataFrame(Y).set_index(['tox','dtp','cl','cvk','n_desc','n_obs'])
P21_sd = P21_sd.groupby(level=['tox','dtp','cl','cvk','n_desc','n_obs']).mean()


In [22]:
Table5 = buildMSTable2(P21_mn,P21_sd)
Table5


n_desc                     bacc                            \
             bio   chm    bc          bio          chm           bc   
tox cl                                                                
Hyp CART0   55.0  65.0  65.0  0.80 (0.09)  0.80 (0.10)  0.81 (0.09)   
    ENSMB   50.0  65.0  50.0  0.81 (0.08)  0.77 (0.09)  0.81 (0.10)   
    KNN1    50.0  65.0  55.0  0.73 (0.12)  0.74 (0.10)  0.77 (0.11)   
    LDA     50.0  65.0  35.0  0.77 (0.09)  0.73 (0.09)  0.79 (0.10)   
    NB      25.0  50.0  50.0  0.71 (0.09)  0.70 (0.09)  0.75 (0.11)   
    SVCL0   60.0  65.0  55.0  0.78 (0.08)  0.73 (0.09)  0.79 (0.11)   
    SVCR0   20.0  60.0  25.0  0.78 (0.08)  0.74 (0.09)  0.78 (0.09)   
Inj CART0   65.0  45.0  40.0  0.76 (0.12)  0.77 (0.11)  0.78 (0.10)   
    ENSMB   65.0  60.0  30.0  0.78 (0.11)  0.75 (0.11)  0.75 (0.10)   
    KNN1    40.0  40.0  45.0  0.68 (0.12)  0.73 (0.13)  0.72 (0.12)   
    LDA     65.0  65.0  25.0  0.73 (0.12)  0.73 (0.12)  0.74 (0.09)   
    NB      60.0  10.0  50.0  0.69 (0.11)  0.68 (0.11)  0.69 (0.10)   
    SVCL0   60.0  65.0  45.0  0.74 (0.11)  0.72 (0.12)  0.74 (0.09)   
    SVCR0   65.0  65.0  60.0  0.78 (0.11)  0.74 (0.12)  0.74 (0.11)   
Pro CART0   20.0  65.0  65.0  0.72 (0.12)  0.76 (0.12)  0.75 (0.10)   
    ENSMB   60.0  65.0  65.0  0.74 (0.12)  0.75 (0.12)  0.75 (0.11)   
    KNN1    55.0  65.0  50.0  0.66 (0.12)  0.71 (0.12)  0.71 (0.14)   
    LDA     65.0  60.0  65.0  0.69 (0.12)  0.73 (0.12)  0.71 (0.11)   
    NB      10.0  45.0  35.0  0.62 (0.11)  0.70 (0.12)  0.68 (0.12)   
    SVCL0   65.0  60.0  60.0  0.69 (0.13)  0.74 (0.12)  0.71 (0.12)   
    SVCR0   65.0  65.0  65.0  0.74 (0.12)  0.74 (0.12)  0.73 (0.10)   

                  sens                                   spec               \
                   bio          chm           bc          bio          chm   
tox cl                                                                       
Hyp CART0  0.75 (0.13)  0.82 (0.14)  0.82 (0.16)  0.87 (0.11)  0.79 (0.14)   
    ENSMB  0.68 (0.14)  0.73 (0.13)  0.76 (0.16)  0.94 (0.08)  0.81 (0.13)   
    KNN1   0.76 (0.15)  0.81 (0.16)  0.79 (0.16)  0.71 (0.25)  0.74 (0.20)   
    LDA    0.70 (0.15)  0.70 (0.12)  0.74 (0.16)  0.88 (0.11)  0.80 (0.14)   
    NB     0.69 (0.26)  0.78 (0.19)  0.77 (0.19)  0.94 (0.33)  0.85 (0.25)   
    SVCL0  0.70 (0.14)  0.69 (0.13)  0.76 (0.16)  0.89 (0.10)  0.79 (0.14)   
    SVCR0  0.84 (0.21)  0.76 (0.19)  0.85 (0.21)  0.97 (0.22)  0.79 (0.19)   
Inj CART0  0.73 (0.19)  0.81 (0.20)  0.81 (0.18)  0.85 (0.15)  0.79 (0.16)   
    ENSMB  0.66 (0.17)  0.73 (0.18)  0.71 (0.19)  0.91 (0.14)  0.81 (0.15)   
    KNN1   0.73 (0.19)  0.79 (0.24)  0.77 (0.21)  0.70 (0.24)  0.72 (0.32)   
    LDA    0.71 (0.16)  0.72 (0.18)  0.73 (0.19)  0.80 (0.16)  0.80 (0.16)   
    NB     0.63 (0.23)  0.83 (0.23)  0.79 (0.30)  0.90 (0.24)  0.81 (0.28)   
    SVCL0  0.67 (0.18)  0.69 (0.17)  0.74 (0.17)  0.84 (0.15)  0.80 (0.16)   
    SVCR0  0.64 (0.18)  0.69 (0.21)  0.75 (0.23)  0.96 (0.21)  0.89 (0.20)   
Pro CART0  0.69 (0.19)  0.79 (0.20)  0.78 (0.20)  0.82 (0.20)  0.74 (0.20)   
    ENSMB  0.66 (0.21)  0.75 (0.21)  0.69 (0.20)  0.82 (0.17)  0.76 (0.20)   
    KNN1   0.76 (0.24)  0.89 (0.25)  0.80 (0.23)  0.60 (0.29)  0.64 (0.34)   
    LDA    0.70 (0.20)  0.75 (0.21)  0.71 (0.19)  0.80 (0.18)  0.72 (0.19)   
    NB     0.78 (0.29)  0.84 (0.23)  0.70 (0.21)  0.78 (0.33)  0.78 (0.24)   
    SVCL0  0.64 (0.20)  0.76 (0.21)  0.73 (0.19)  0.81 (0.17)  0.72 (0.22)   
    SVCR0  0.63 (0.22)  0.73 (0.21)  0.71 (0.24)  0.93 (0.23)  0.84 (0.21)   

                        
                    bc  
tox cl                  
Hyp CART0  0.84 (0.13)  
    ENSMB  0.88 (0.11)  
    KNN1   0.78 (0.25)  
    LDA    0.84 (0.11)  
    NB     0.83 (0.21)  
    SVCL0  0.83 (0.13)  
    SVCR0  0.94 (0.21)  
Inj CART0  0.79 (0.18)  
    ENSMB  0.82 (0.16)  
    KNN1   0.69 (0.26)  
    LDA    0.80 (0.15)  
    NB     0.68 (0.32)  
    SVCL0  0.80 (0.16)  
    SVCR0  0.91 (0.21)  
Pro CAR